**сеть Хемминга**

нечёткий поиск по словарю

Евгений Борисов esborisov@sevsu.ru

---

Борисов Е.  Классификатор на основе нейронной сети Хемминга.   
http://mechanoid.su/neural-net-hamming-classifier.html

Арустамов А., Стариков А.   Ассоциативная память.Применение сетей Хемминга для нечеткого поиска.   
https://basegroup.ru/community/articles/assoc

##  загружаем данные 

In [1]:
import gzip
from random import sample

with gzip.open('data/ideal_u.txt.gz','rt') as f: 
    ideal = [ w.strip() for w in f.read().split('\n') if w.strip() ]
        
display(len(ideal))
display(sample(ideal,5))

112

['Одинцово', 'Новгород', 'Каунас', 'Гомель', 'Пенза']

## кодируем данные

In [2]:
import numpy as np
from numpy import random as rng

In [4]:
class DataEncoder:
    
    def __init__(self):
        # Кодирование строится таким образом, 
        # что бы стоящие рядом на компьютерной клавиатуре символы 
        # имели близкие по Хеммингу коды.
        # Таким образом должно достигаться наиболее эффективное исправление опечаток.
        self._abc = {
           'к':(0,0,0,0,0,),
           'ж':(0,0,0,0,1,),
           'г':(0,0,0,1,0,),
           'щ':(0,0,0,1,1,),
           'ч':(0,0,1,0,0,),
           'х':(0,0,1,0,1,),
           'ю':(0,0,1,1,0,),
           'с':(0,0,1,1,1,),
           'у':(0,1,0,0,0,),
           'б':(0,1,0,0,1,),
           'л':(0,1,0,1,0,),
           'е':(0,1,0,1,1,),
           'о':(0,1,1,0,0,),
           'р':(0,1,1,0,1,),
           'н':(0,1,1,1,0,),
           'й':(0,1,1,1,1,),
           'т':(1,0,0,0,0,),
           'з':(1,0,0,0,1,),
           'ы':(1,0,0,1,0,),
           'п':(1,0,0,1,1,),
           'ф':(1,0,1,0,0,),
           'ш':(1,0,1,0,1,),
           'м':(1,0,1,1,0,),
           'э':(1,0,1,1,1,),
           'ъ':(1,1,0,0,0,),
           'д':(1,1,0,0,1,),
           'в':(1,1,0,1,0,),
           'а':(1,1,0,1,1,),
           'ц':(1,1,1,0,0,),
           'и':(1,1,1,0,1,),
           'я':(1,1,1,1,0,),
           'ь':(1,1,1,1,1,),
        }

    # собираем датасет из списка слов text,
    # max_word - ограничение максимальной длины слова (0 - нет ограничений)
    def transform(self,text,max_word_len=0):
        # заменяем буквы на коды
        c = [ self._encode(w) for w in text ] 
        
        x = np.array( self._pad(c,max_word_len) )*2-1
        n_samples, seq_len, code_len = x.shape
        return x.reshape(n_samples,seq_len*code_len)

    def _encode(self,w):
        return  [ self._abc[a] for a in w.lower() ]
   
    # дополняем коды коротких слов нулями
    def _pad(self,x,max_word_len): 
        code_len = len(x[0][0]) # длинна кода
        # максимальная длинна слова
        mwl = max([len(v) for v in x]+[max_word_len])
        # дополнение нулями
        z = [(0,)*code_len]*mwl
        # дополняем короткое слово
        return [ v + z[:(mwl-len(v))] for v in x ]

In [5]:
x_train = DataEncoder().transform(ideal)
display(x_train.shape)

(112, 70)

In [6]:
x_train

array([[ 1,  1, -1, ..., -1, -1, -1],
       [ 1,  1, -1, ..., -1, -1, -1],
       [ 1,  1, -1, ..., -1, -1, -1],
       ...,
       [ 1, -1,  1, ..., -1, -1, -1],
       [ 1,  1,  1, ..., -1, -1, -1],
       [ 1,  1,  1, ..., -1, -1, -1]])

# модель

сеть Хэминга

![neural-net-hamming](http://mechanoid.su/content/neural-net-hamming-classifier.html/nnet.png)

In [7]:
class HammingNet:
    
    def __init__(self):
        self._weight_lin = 0.
        self._weight_hop = 0.
        
    def fit(self,x):
        self._weight_lin =  0.5*x.T
        
        n_samples,_ = x.shape # количество учебных примеров 
        c = 1./(2.*n_samples) # коэффициент торможения
        # веса для нейронов второго слоя
        self._weight_hop = -c*(np.ones(n_samples)-np.eye(n_samples)) + np.eye(n_samples)
        
        return self
    
    def forward(self,x,max_iter=8):
        o = x.dot(self._weight_lin)
        for n in range(max_iter):
            o_ = o.copy() # сохраняем состояние
            o = self._forward_step(o) # переходим в новое состояние
            # если состояние не изменилось то завершаем
            if np.all(o==o_): break
        return n,o        
    
    def _forward_step(self,x):
        return np.max( [x.dot(self._weight_hop), np.zeros(x.shape)],axis=0 )
    
    def __call__(self,x):
        return self.forward(x)
    
    def predict(self,x):
        n,o = self.forward(x)
        return n,np.argmax(o,axis=1)

# загружаем память модели

In [8]:
model = HammingNet().fit(x_train)

## тестируем

In [9]:
with gzip.open('data/test_u.txt.gz','rt') as f: 
    test = [ w.strip() for w in f.read().split('\n') if w.strip() ]
        
display(len(test))
display(sample(test,5))

112

['Ньвгьрьд', 'Тумюьв', 'Тьмск', 'Кцрск', 'Бурнуул']

In [10]:
max_word_len = max([ len(w) for w in ideal ])
display(max_word_len)

14

In [11]:
x_test = DataEncoder().transform(test,max_word_len=max_word_len)
display(x_test.shape)

(112, 70)

In [12]:
from tabulate import tabulate 

_,p = model.predict(x_test)
res = [ 
        [ t, ideal[ p[i] ] ] 
        for i,t in enumerate(test) 
    ]

tabulate(res, tablefmt='html')

Акмьлу,Актау
Актуу,Актау
Алмуты,Алматы
Архунгельск,Архангельск
Аструхунь,Астрахань
Ашхуюуд,Ашхабад
Буку,Баку
Бурнуул,Барнаул
Билкек,Бишкек
Блугьвещенск,Благовещенск
Брутск,Братск
